In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


***Step 1. Schema Validation***

Description

In this step, I performed a schema validation check. The goal was to ensure that the columns in the Bronze layer tables (sales and rebates) matched the expected structure defined for the Silver layer. This helps catch issues early, like missing columns or unexpected changes in the raw files.

Key Points

Defined a set of expected columns for both Sales and Rebate datasets.

Retrieved the actual column names present in the Bronze tables.

Compared expected vs. actual to find any missing or mismatched columns.

Printed out any differences for manual review and debugging.

Ensures that downstream Silver transformations won’t break due to schema issues.

In [1]:
#Step 1. Schema Validation
# Expected schemas
expected_sales_cols = {
    "sale_transaction_id","customer_id","customer_name","customer_country",
    "customer_region","customer_country_code",
    "sales_rep_id","sales_rep_name","sales_rep_country","sales_rep_region","sales_rep_country_code",
    "product_code","product_details",
    "unit_quantity","sale_amount","sale_datetime",
    "sale_location","currencycode","sales_country_code"
}

expected_rebates_cols = {
    "rebate_transaction_id","sale_transaction_id","customer_id","sales_rep_id",
    "rebate_type","rebate_amount","rebate_datetime","sale_location","currencycode"
}

# Compare with Bronze
bronze_sales_cols = set(spark.table("bronze_sales_raw").columns)
bronze_rebates_cols = set(spark.table("bronze_rebates_raw").columns)

print("Missing in Sales:", expected_sales_cols - bronze_sales_cols)
print("Missing in Rebates:", expected_rebates_cols - bronze_rebates_cols)



StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 3, Finished, Available, Finished)

Missing in Sales: set()
Missing in Rebates: set()


***#Step 2. Full Data Load***

In this I loaded the Bronze tables (bronze_sales_raw, bronze_rebates_raw) into Spark DataFrames for further cleaning and transformation in the Silver layer.

In [2]:
df_sales_bronze = spark.read.table("bronze_sales_raw")
df_rebates_bronze = spark.read.table("bronze_rebates_raw")

StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 4, Finished, Available, Finished)

***# Step 3. Transformtions***

***#Step 3.1. Schema & Types***

***

Description

In this step, I transformed the Bronze DataFrames into Silver-ready DataFrames by applying type casting, date handling, and standardization. This ensures the data is accurate, consistent, and business-friendly.

Key Points

Type Casting: Converted numbers (unit_quantity, sale_amount, rebate_amount) into proper numeric types and parsed date/timestamp fields.

Derived Columns: Extracted sale_date and rebate_date for easier reporting.

Standardized Values: Converted currencycode to uppercase for consistency.

Cleaned Text Fields: Trimmed spaces and applied proper casing (e.g., Customer Name, Sales Rep Name, Product Details, Sale Location).

Prepared Clean Silver DataFrames: Created df_sales_silver and df_rebates_silver ready for loading into Silver tables.

In [3]:
from pyspark.sql import functions as F

# Sales Transform
df_sales_silver = (
    df_sales_bronze
    # Type casting
    .withColumn("unit_quantity", F.col("unit_quantity").cast("int"))
    .withColumn("sale_amount", F.col("sale_amount").cast("decimal(18,2)"))
    .withColumn("sale_datetime", F.to_timestamp("sale_datetime"))
    .withColumn("sale_date", F.to_date("sale_datetime"))
    # Standardization
    .withColumn("currencycode", F.upper(F.col("currencycode")))
    .withColumn("customer_name", F.initcap(F.trim(F.col("customer_name"))))
    .withColumn("sales_rep_name", F.initcap(F.trim(F.col("sales_rep_name"))))
    .withColumn("product_details", F.trim(F.col("product_details")))
    .withColumn("sale_location", F.trim(F.col("sale_location")))
)

# Rebates Transform
df_rebates_silver = (
    df_rebates_bronze
    .withColumn("rebate_amount", F.col("rebate_amount").cast("decimal(18,2)"))
    .withColumn("rebate_datetime", F.to_timestamp("rebate_datetime"))
    .withColumn("rebate_date", F.to_date("rebate_datetime"))
    .withColumn("currencycode", F.upper(F.col("currencycode")))
    .withColumn("sale_location", F.trim(F.col("sale_location")))
)


StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 5, Finished, Available, Finished)

***#Step 3.2. Product Split (Product Details → Segment, Category, Subcategory, Item)***

Description

In this step, I split the unstructured product_details column into multiple structured columns (segment, category, subcategory, and item). This makes it easier to group, filter, and analyze product-level insights in the Silver and Gold layers.

Key Points

Used split on the product_details field (delimited by |).

Extracted Segment (level 1), Category (level 2), Subcategory (level 3), and Item (level 4).

Added these as new columns in df_sales_silver.

Turned product data from a single string into structured attributes.

Enables hierarchical analysis of sales by product breakdown.

In [4]:
df_sales_silver = (
    df_sales_silver
    .withColumn("segment", F.split("product_details", "\\|").getItem(0))
    .withColumn("category", F.split("product_details", "\\|").getItem(1))
    .withColumn("subcategory", F.split("product_details", "\\|").getItem(2))
    .withColumn("item", F.split("product_details", "\\|").getItem(3))
)

StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 6, Finished, Available, Finished)

***#Step 3.3. Product Code Parse**

Description

In this step, I decomposed the product_code column into multiple levels, creating structured fields for easier reporting and analysis.

Key Points

Split the product_code field (delimited by |).

Extracted code_l1, code_l2, code_l3, and sku.

Added these new columns into df_sales_silver.

Converted product codes from one long string into separate attributes.

This allows drill-down analysis at different product code levels.

In [5]:
df_sales_silver = (
    df_sales_silver
    .withColumn("code_l1", F.split("product_code", "\\|").getItem(0))
    .withColumn("code_l2", F.split("product_code", "\\|").getItem(1))
    .withColumn("code_l3", F.split("product_code", "\\|").getItem(2))
    .withColumn("sku", F.split("product_code", "\\|").getItem(3))
)

StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 7, Finished, Available, Finished)

***# Step 3.4. Countries/Regions Normalization
***

Description

In this step, I created a reference country dimension in Silver and used it to assign surrogate keys for both customer countries and sales rep countries. This helps standardize country information and makes joins more reliable.

Key Points

Built a country dimension table (silver_dim_country) with country name, ISO code, region, and a surrogate key.

Cleaned values by trimming spaces, fixing letter case, and formatting consistently.

Generated a surrogate key (country_sk) for each country.

Joined this dimension to customer_country in sales data to assign customer_country_sk.

Joined it again to sales_rep_country in sales data to assign salesrep_country_sk.

In [13]:
# Step 3.4 Create Reference Dimensions in Silver

# Create silver_dim_country once
dim_country_data = [
    ("United States", "US", "North America"),
    ("Germany", "DE", "EMEA"),
    ("France", "FR", "EMEA"),
    ("United Kingdom", "UK", "EMEA"),
    ("Mexico", "MX", "North America"),
    ("Canada", "CA", "North America"),
    ("India", "IN", "APAC"),
    ("Australia", "AU", "APAC")
]

columns = ["country_name","iso2","region"]

df_country = spark.createDataFrame(dim_country_data, columns)

from pyspark.sql import functions as F
from pyspark.sql.window import Window

df_country_silver = (
    df_country
    .withColumn("country_name", F.initcap(F.trim(F.col("country_name"))))
    .withColumn("iso2", F.upper(F.trim(F.col("iso2"))))
    .withColumn("region", F.initcap(F.trim(F.col("region"))))
    .withColumn("country_sk", F.row_number().over(Window.orderBy("country_name")))
)

spark.sql("""
CREATE TABLE IF NOT EXISTS silver_dim_country (
  country_name STRING,
  iso2 STRING,
  region STRING,
  country_sk BIGINT
)
""")

df_country_silver.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("silver_dim_country")



# Use dim_country to normalize and assign surrogate keys
#dim_country = spark.table("dim_country").select("country_name", "iso2", "region", "country_sk")

dim_country = spark.table("silver_dim_country").select("country_name", "iso2", "region", "country_sk")

# Customer country SK
df_sales_silver = (
    df_sales_silver
    .join(
        dim_country.withColumnRenamed("country_name", "cust_country_name")
                   .withColumnRenamed("country_sk", "cust_country_sk"),
        df_sales_silver.customer_country == F.col("cust_country_name"),
        "left"
    )
    .withColumn("customer_country_sk", F.col("cust_country_sk"))
    .drop("cust_country_name", "cust_country_sk", "iso2", "region")
)

# Sales rep country SK
df_sales_silver = (
    df_sales_silver
    .join(
        dim_country.withColumnRenamed("country_name", "rep_country_name")
                   .withColumnRenamed("country_sk", "rep_country_sk"),
        df_sales_silver.sales_rep_country == F.col("rep_country_name"),
        "left"
    )
    .withColumn("salesrep_country_sk", F.col("rep_country_sk"))
    .drop("rep_country_name", "rep_country_sk", "iso2", "region")
)




StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 15, Finished, Available, Finished)

***#Step 3.5. Sale Location Standardization***

Description

In this step, I split the sale_location field into two separate columns so the region and country are stored clearly instead of in one combined string.

Key Points

Took the sale_location column (formatted as Region|Country).

Split it into sale_region (first part) and sale_country (second part).

Added these two new columns into df_sales_silver.

This makes filtering and grouping by region or country much easier.

Avoids confusion from having both values stored together in one field.

In [14]:
# Example assumes format "Region|Country"
df_sales_silver = (
    df_sales_silver
    .withColumn("sale_region", F.split("sale_location", "\\|").getItem(0))
    .withColumn("sale_country", F.split("sale_location", "\\|").getItem(1))
)

StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 16, Finished, Available, Finished)

***#Step 3.6. Currency Enhancements***

Description

In this step, I added checks and flags for currency values and also created a new column to show sales amounts converted into USD.

Key Points

Validated currencies: Marked each row with is_valid_currency (1 if allowed, 0 if not).

Cross-currency flag: Created is_cross_currency to highlight when a US sale was recorded in a non-USD currency.

FX conversion: Used a sample exchange rate mapping to convert sale_amount into sale_amount_usd.

Prepared for future enhancement: In production, this should connect to a live FX API for accurate conversion.

Ensures sales amounts are standardized and comparable across regions.

In [15]:
# Currency validation
valid_currencies = ["USD","EUR","GBP","INR","AUD","CAD"]
df_sales_silver = df_sales_silver.withColumn(
    "is_valid_currency",
    F.when(F.col("currencycode").isin(valid_currencies), F.lit(1)).otherwise(F.lit(0))
)

# Cross-currency flag (if currency mismatches country’s standard currency)
df_sales_silver = df_sales_silver.withColumn(
    "is_cross_currency",
    F.when((F.col("currencycode") != "USD") & (F.col("sales_country_code") == "US"), F.lit(1))
     .otherwise(F.lit(0))
)

# Example FX conversion (placeholder, ideally use FX rates table / API)
fx_rates = {"USD":1.0,"EUR":1.1,"GBP":1.3,"INR":0.012,"AUD":0.7,"CAD":0.75}
fx_map = F.create_map([F.lit(x) for x in sum(fx_rates.items(), ())])

df_sales_silver = df_sales_silver.withColumn(
    "sale_amount_usd",
    (F.col("sale_amount") / fx_map[F.col("currencycode")]).cast("decimal(18,2)")
)


StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 17, Finished, Available, Finished)

***#Step 4. Silver Fact Staging (CREATE + LOAD)***

Description

In this step, I created the fact staging tables for sales and rebates in the Silver layer and loaded the cleaned data into them. These tables hold transactional records in a structured and consistent format, ready for further processing in the Gold layer.

Key Points

Defined silver_fact_sales_stage with proper data types (numeric, date, timestamp).

Defined silver_fact_rebates_stage for rebate transactions.

Loaded cleaned Sales data into silver_fact_sales_stage.

Loaded cleaned Rebates data into silver_fact_rebates_stage.

Used overwrite mode so the tables always refresh with the latest cleaned data.

In [16]:
# Create Fact tables
spark.sql("""
CREATE TABLE IF NOT EXISTS silver_fact_sales_stage (
  sale_transaction_id STRING,
  customer_id STRING, customer_name STRING, customer_country STRING,
  customer_region STRING, customer_country_code STRING,
  sales_rep_id STRING, sales_rep_name STRING, sales_rep_country STRING,
  sales_rep_region STRING, sales_rep_country_code STRING,
  product_code STRING, product_details STRING,
  unit_quantity INT, sale_amount DECIMAL(18,2),
  sale_datetime TIMESTAMP, sale_date DATE,
  sale_location STRING, currencycode STRING, sales_country_code STRING
)
""")

spark.sql("""
CREATE TABLE IF NOT EXISTS silver_fact_rebates_stage (
  rebate_transaction_id STRING, sale_transaction_id STRING,
  customer_id STRING, sales_rep_id STRING,
  rebate_type STRING, rebate_amount DECIMAL(18,2),
  rebate_datetime TIMESTAMP, rebate_date DATE,
  currencycode STRING, sale_location STRING
)
""")

# Load into Fact Staging
df_sales_silver.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("silver_fact_sales_stage")
df_rebates_silver.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("silver_fact_rebates_stage")

StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 18, Finished, Available, Finished)

***# Step 5: Run DQ***

Description

In this step, I created a process to check the quality of the data in the Silver fact staging tables (sales and rebates). I defined rules to catch errors like negative amounts, missing IDs, invalid currencies, and duplicates. Any issues found are logged into the silver_dq_issues table with details for monitoring.

Key Points

Created the silver_dq_issues table to store DQ results.

Added a unique run ID and timestamp to track when checks were done.

Checked sales data for: negative amounts, zero quantities, missing IDs, duplicates, and invalid currencies.

Checked rebates data for: negative amounts, missing/duplicate IDs, and orphan rebates (rebates without a matching sale).

Logged all findings with severity levels (HIGH or MEDIUM) into the issues table.

In [17]:
import uuid, datetime
from pyspark.sql import functions as F

# ensure table exists
spark.sql("""
CREATE TABLE IF NOT EXISTS silver_dq_issues (
  run_id STRING, table_name STRING, rule_name STRING,
  severity STRING, issue_count BIGINT, checked_at TIMESTAMP
)
""")

run_id = str(uuid.uuid4())
checked_at = datetime.datetime.utcnow()
dq_results = []

df_sales   = spark.table("silver_fact_sales_stage")
df_rebates = spark.table("silver_fact_rebates_stage")

# Sales rules
c = df_sales.filter(F.col("sale_amount") <= 0).count()
if c: dq_results.append((run_id,"silver_fact_sales_stage","sale_amount_le_zero","HIGH",c,checked_at))

c = df_sales.filter(F.col("unit_quantity") <= 0).count()
if c: dq_results.append((run_id,"silver_fact_sales_stage","unit_quantity_le_zero","HIGH",c,checked_at))

c = df_sales.filter(
    F.col("sale_transaction_id").isNull() | F.col("customer_id").isNull() | F.col("sales_rep_id").isNull()
).count()
if c: dq_results.append((run_id,"silver_fact_sales_stage","null_critical_fields","HIGH",c,checked_at))

c = df_sales.groupBy("sale_transaction_id").count().filter("count > 1").count()
if c: dq_results.append((run_id,"silver_fact_sales_stage","duplicate_sale_transaction_id","HIGH",c,checked_at))

valid_currencies = ["USD","EUR","GBP","INR","AUD","CAD"]
c = df_sales.filter(~F.col("currencycode").isin(valid_currencies)).count()
if c: dq_results.append((run_id,"silver_fact_sales_stage","invalid_currencycode","MEDIUM",c,checked_at))

# Rebates rules
c = df_rebates.filter(F.col("rebate_amount") <= 0).count()
if c: dq_results.append((run_id,"silver_fact_rebates_stage","rebate_amount_le_zero","HIGH",c,checked_at))

c = df_rebates.filter(F.col("rebate_transaction_id").isNull()).count()
if c: dq_results.append((run_id,"silver_fact_rebates_stage","null_rebate_transaction_id","HIGH",c,checked_at))

c = df_rebates.groupBy("rebate_transaction_id").count().filter("count > 1").count()
if c: dq_results.append((run_id,"silver_fact_rebates_stage","duplicate_rebate_transaction_id","HIGH",c,checked_at))

c = df_rebates.join(df_sales, "sale_transaction_id", "left_anti").count()
if c: dq_results.append((run_id,"silver_fact_rebates_stage","orphan_rebate","MEDIUM",c,checked_at))

if dq_results:
    dq_df = spark.createDataFrame(dq_results, """
        run_id string, table_name string, rule_name string,
        severity string, issue_count long, checked_at timestamp
    """)
    dq_df.write.mode("append").saveAsTable("silver_dq_issues")

    

StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 19, Finished, Available, Finished)

Description

In this step, I expanded the data quality process to capture not only summary counts of issues but also detailed row-level information. By storing both summary and detailed results, I can monitor overall data health and also trace exactly which records failed specific rules.

Key Points

Created two logging tables:

silver_dq_issues → summary of issues (rule name, count, severity).

silver_dq_issues_details → detailed row-level information for each failed record.

Added a helper function to automatically log both summary and details for each rule.

Checked sales data for negative/zero amounts, missing critical IDs, duplicates, and invalid currency codes.

Checked rebates data for negative amounts, missing or duplicate IDs, and orphan rebates (rebates without matching sales).

Saved results into the summary and details tables with run IDs and timestamps for full traceability.

In [5]:
import uuid, datetime
from pyspark.sql import functions as F

# --------------------------------
# 1. Create Tables (if not exist)
# --------------------------------

# Summary table
spark.sql("""
CREATE TABLE IF NOT EXISTS silver_dq_issues (
  run_id STRING,
  table_name STRING,
  rule_name STRING,
  severity STRING,
  issue_count BIGINT,
  checked_at TIMESTAMP
)
""")

# Detail table
spark.sql("""
CREATE TABLE IF NOT EXISTS silver_dq_issues_details (
  run_id STRING,
  table_name STRING,
  rule_name STRING,
  severity STRING,
  column_name STRING,
  primary_key STRING,
  actual_value STRING,
  expected_value STRING,
  checked_at TIMESTAMP
)
""")

# --------------------------------
# 2. Setup
# --------------------------------
run_id = str(uuid.uuid4())
checked_at = datetime.datetime.utcnow()

df_sales   = spark.table("silver_fact_sales_stage")
df_rebates = spark.table("silver_fact_rebates_stage")

summary_results = []
detail_dfs = []   # collect DataFrames for detailed issues

# --------------------------------
# 3. Helper Function
# --------------------------------
def log_issue_with_details(df, table, rule, severity, col_name, pk_col, condition, expected_value):
    bad_df = df.filter(condition)
    c = bad_df.count()
    if c > 0:
        # --- summary ---
        summary_results.append((run_id, table, rule, severity, c, checked_at))

        # --- details ---
        detail_df = (bad_df
            .withColumn("run_id", F.lit(run_id))
            .withColumn("table_name", F.lit(table))
            .withColumn("rule_name", F.lit(rule))
            .withColumn("severity", F.lit(severity))
            .withColumn("column_name", F.lit(col_name))
            .withColumn("primary_key", F.col(pk_col).cast("string"))
            .withColumn("actual_value", F.col(col_name).cast("string"))
            .withColumn("expected_value", F.lit(expected_value))
            .withColumn("checked_at", F.lit(checked_at))
            .select(  #  enforce consistent schema
                "run_id","table_name","rule_name","severity",
                "column_name","primary_key","actual_value","expected_value","checked_at"
            )
        )
        detail_dfs.append(detail_df)

# --------------------------------
# 4. SALES RULES
# --------------------------------
log_issue_with_details(df_sales, "silver_fact_sales_stage", "sale_amount_le_zero",
                       "HIGH", "sale_amount", "sale_transaction_id",
                       F.col("sale_amount") <= 0, "> 0")

log_issue_with_details(df_sales, "silver_fact_sales_stage", "unit_quantity_le_zero",
                       "HIGH", "unit_quantity", "sale_transaction_id",
                       F.col("unit_quantity") <= 0, "> 0")

log_issue_with_details(df_sales, "silver_fact_sales_stage", "null_critical_fields",
                       "HIGH", "customer_id", "sale_transaction_id",
                       F.col("sale_transaction_id").isNull() |
                       F.col("customer_id").isNull() |
                       F.col("sales_rep_id").isNull(),
                       "Not Null")

dup_sales = (df_sales.groupBy("sale_transaction_id").count()
                       .filter("count > 1")
                       .select("sale_transaction_id"))
log_issue_with_details(df_sales.join(dup_sales, "sale_transaction_id"),
                       "silver_fact_sales_stage", "duplicate_sale_transaction_id",
                       "HIGH", "sale_transaction_id", "sale_transaction_id",
                       F.lit(True), "Unique")

valid_currencies = ["USD","EUR","GBP","INR","AUD","CAD"]
log_issue_with_details(df_sales, "silver_fact_sales_stage", "invalid_currencycode",
                       "MEDIUM", "currencycode", "sale_transaction_id",
                       ~F.col("currencycode").isin(valid_currencies),
                       f"Valid in {valid_currencies}")

# --------------------------------
# 5. REBATES RULES
# --------------------------------
log_issue_with_details(df_rebates, "silver_fact_rebates_stage", "rebate_amount_le_zero",
                       "HIGH", "rebate_amount", "rebate_transaction_id",
                       F.col("rebate_amount") <= 0, "> 0")

log_issue_with_details(df_rebates, "silver_fact_rebates_stage", "null_rebate_transaction_id",
                       "HIGH", "rebate_transaction_id", "rebate_transaction_id",
                       F.col("rebate_transaction_id").isNull(), "Not Null")

dup_rebates = (df_rebates.groupBy("rebate_transaction_id").count()
                          .filter("count > 1")
                          .select("rebate_transaction_id"))
log_issue_with_details(df_rebates.join(dup_rebates, "rebate_transaction_id"),
                       "silver_fact_rebates_stage", "duplicate_rebate_transaction_id",
                       "HIGH", "rebate_transaction_id", "rebate_transaction_id",
                       F.lit(True), "Unique")

log_issue_with_details(df_rebates.join(df_sales, "sale_transaction_id", "left_anti"),
                       "silver_fact_rebates_stage", "orphan_rebate",
                       "MEDIUM", "sale_transaction_id", "rebate_transaction_id",
                       F.lit(True), "Must match sales")

# --------------------------------
# 6. Save Results
# --------------------------------

# Save summary table
if summary_results:
    dq_summary_df = spark.createDataFrame(summary_results, """
        run_id string, table_name string, rule_name string,
        severity string, issue_count long, checked_at timestamp
    """)
    dq_summary_df.write.mode("append").saveAsTable("silver_dq_issues")

# Save details table
if detail_dfs:
    dq_details_df = detail_dfs[0]
    for d in detail_dfs[1:]:
        dq_details_df = dq_details_df.unionByName(d)
    dq_details_df.write.mode("append").saveAsTable("silver_dq_issues_details")


StatementMeta(, c3ead005-5404-4e7c-b782-7bcd72ee47f8, 7, Finished, Available, Finished)

***#Step 6. Build Dimension Snapshots → SCD2 + Surrogate Keys***

Description

In this step, I implemented a generic Slowly Changing Dimension Type 2 (SCD2) function to manage changes in dimension tables such as Customer, SalesRep, Product, and Geo. This ensures that when attributes change (like a sales rep moving to a new region), historical records are preserved while marking the most recent version as current.

Key Points

Built a reusable function (scd2_upsert) that works for any dimension table.

Compares the incoming snapshot (source_df) with the existing dimension table using a business key (e.g., customer_id).

Detects changes by comparing tracked attributes (e.g., name, region, country).

When a change is found, the old record is closed (valid_to updated, is_current=0) and a new record is inserted with updated values.

Maintains full history of changes while keeping the latest version marked as is_current=1.

In [19]:
from pyspark.sql import Window

StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 21, Finished, Available, Finished)

In [20]:
from pyspark.sql import functions as F

def scd2_upsert(source_df,
                target_table: str,
                business_key: str,
                attributes: list,
                change_ts_col: str):
    """
    Generic SCD2 upsert function for slowly changing dimensions (Type 2).
    
    Args:
        source_df (DataFrame): Incoming snapshot with business key, attributes, and change timestamp.
        target_table (str): Name of the Delta/Spark table (e.g., "silver_dim_customer_scd2").
        business_key (str): Column name used as business key (e.g., "customer_id").
        attributes (list): List of attribute column names to track changes.
        change_ts_col (str): Column name in source_df that has the change timestamp.
    """

    tmp_view = f"tmp_{business_key}_scd2"
    source_df.createOrReplaceTempView(tmp_view)

    change_conditions = " OR ".join([f"t.{col} <> s.{col}" for col in attributes])

    merge_sql = f"""
    MERGE INTO {target_table} t
    USING {tmp_view} s
    ON t.{business_key} = s.{business_key} AND t.is_current = 1
    WHEN MATCHED AND ({change_conditions})
      THEN UPDATE SET t.valid_to = s.{change_ts_col}, t.is_current = 0
    WHEN NOT MATCHED
      THEN INSERT ({business_key}, {",".join(attributes)}, valid_from, valid_to, is_current)
           VALUES (s.{business_key}, {",".join([f"s.{col}" for col in attributes])},
                   s.{change_ts_col}, TIMESTAMP'9999-12-31', 1)
    """

    spark.sql(merge_sql)
    print(f"SCD2 upsert completed for {target_table}")


StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 22, Finished, Available, Finished)

In this step, I applied **Slowly Changing Dimension Type 2 (SCD2)** to Customers, Sales Reps, Products, and Geo. Instead of overwriting old details when something changes (like a sales rep moving to a new region), I kept the old record and added a new one with updated values. This way, the history is preserved while the latest version is always marked as current, making reporting more accurate over time.


In [21]:
from pyspark.sql import Window

#1. Customer Snapshot
w_cust = Window.partitionBy("customer_id").orderBy(F.col("sale_datetime").desc())
cust_snapshot = (
    spark.table("silver_fact_sales_stage")
    .withColumn("rn", F.row_number().over(w_cust))
    .filter("rn = 1")
    .select("customer_id","customer_name","customer_country","customer_region","customer_country_code",F.col("sale_datetime").alias("change_ts"))
)

# Create dimension table if not exists
spark.sql("""
CREATE TABLE IF NOT EXISTS silver_dim_customer_scd2 (
  customer_id STRING, customer_name STRING, customer_country STRING,
  customer_region STRING, customer_country_code STRING,
  valid_from TIMESTAMP, valid_to TIMESTAMP, is_current INT, customer_sk BIGINT
)
""")

# Apply SCD2
scd2_upsert(
    source_df=cust_snapshot,
    target_table="silver_dim_customer_scd2",
    business_key="customer_id",
    attributes=["customer_name","customer_country","customer_region","customer_country_code"],
    change_ts_col="change_ts"
)

# Regenerate surrogate keys
cust_dim = (
    spark.table("silver_dim_customer_scd2")
    .withColumn("customer_sk", F.row_number().over(Window.orderBy("customer_id","valid_from")))
)
cust_dim.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("silver_dim_customer_scd2")

#2. sales rep snapshot
w_rep = Window.partitionBy("sales_rep_id").orderBy(F.col("sale_datetime").desc())
salesrep_snapshot = (
    spark.table("silver_fact_sales_stage")
    .withColumn("rn", F.row_number().over(w_rep)).filter("rn = 1")
    .select("sales_rep_id","sales_rep_name","sales_rep_country","sales_rep_region","sales_rep_country_code",
            F.col("sale_datetime").alias("change_ts"))
)

spark.sql("""
CREATE TABLE IF NOT EXISTS silver_dim_salesrep_scd2 (
  sales_rep_id STRING, sales_rep_name STRING, sales_rep_country STRING,
  sales_rep_region STRING, sales_rep_country_code STRING,
  valid_from TIMESTAMP, valid_to TIMESTAMP, is_current INT, salesrep_sk BIGINT
)
""")

scd2_upsert(salesrep_snapshot, "silver_dim_salesrep_scd2", "sales_rep_id",
            ["sales_rep_name","sales_rep_country","sales_rep_region","sales_rep_country_code"], "change_ts")

salesrep_dim = (spark.table("silver_dim_salesrep_scd2")
    .withColumn("salesrep_sk", F.row_number().over(Window.orderBy("sales_rep_id","valid_from"))))
salesrep_dim.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("silver_dim_salesrep_scd2")

#3. Product Snapshot
w_prod = Window.partitionBy("product_code").orderBy(F.col("sale_datetime").desc())
product_snapshot = (
    spark.table("silver_fact_sales_stage")
    .withColumn("rn", F.row_number().over(w_prod)).filter("rn = 1")
    .select("product_code","product_details","segment","category","subcategory","item",
            "code_l1","code_l2","code_l3","sku",
            F.col("sale_datetime").alias("change_ts"))
)

spark.sql("""
CREATE TABLE IF NOT EXISTS silver_dim_product_scd2 (
  product_code STRING, product_details STRING,
  segment STRING, category STRING, subcategory STRING, item STRING,
  code_l1 STRING, code_l2 STRING, code_l3 STRING, sku STRING,
  valid_from TIMESTAMP, valid_to TIMESTAMP, is_current INT, product_sk BIGINT
)
""")

scd2_upsert(product_snapshot, "silver_dim_product_scd2", "product_code",
            ["product_details","segment","category","subcategory","item","code_l1","code_l2","code_l3","sku"], "change_ts")

product_dim = (spark.table("silver_dim_product_scd2")
    .withColumn("product_sk", F.row_number().over(Window.orderBy("product_code","valid_from"))))
product_dim.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("silver_dim_product_scd2")
#4 Geo snapshot
w_geo = Window.partitionBy("sales_country_code").orderBy(F.col("sale_datetime").desc())
geo_snapshot = (
    spark.table("silver_fact_sales_stage")
    .withColumn("rn", F.row_number().over(w_geo)).filter("rn = 1")
    .select(F.col("sales_country_code").alias("country_code"),
            F.col("customer_country").alias("country"),
            F.col("customer_region").alias("region"),
            F.col("sale_location").alias("location"),
            F.col("sale_datetime").alias("change_ts"))
)

spark.sql("""
CREATE TABLE IF NOT EXISTS silver_dim_geo_scd2 (
  country_code STRING, country STRING, region STRING, location STRING,
  valid_from TIMESTAMP, valid_to TIMESTAMP, is_current INT, geo_sk BIGINT
)
""")

scd2_upsert(geo_snapshot, "silver_dim_geo_scd2", "country_code",
            ["country","region","location"], "change_ts")

geo_dim = (spark.table("silver_dim_geo_scd2")
    .withColumn("geo_sk", F.row_number().over(Window.orderBy("country_code","valid_from"))))
geo_dim.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("silver_dim_geo_scd2")


StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 23, Finished, Available, Finished)

SCD2 upsert completed for silver_dim_customer_scd2
SCD2 upsert completed for silver_dim_salesrep_scd2
SCD2 upsert completed for silver_dim_product_scd2
SCD2 upsert completed for silver_dim_geo_scd2


***#Step 7. Incremental Load***

In this step, I applied **incremental loading** to the sales and rebates fact staging tables using the `MERGE` command. This approach checks if a record already exists by matching on the transaction ID; if it does, the record is updated with the latest values, and if it doesn’t, a new record is inserted. By doing this, I avoided reprocessing the entire dataset each time and ensured the pipeline only processes new or changed records, making it more efficient and production-ready.


In [22]:
df_sales_silver.createOrReplaceTempView("sales_increment")

spark.sql("""
MERGE INTO silver_fact_sales_stage tgt
USING sales_increment src
ON tgt.sale_transaction_id = src.sale_transaction_id
WHEN MATCHED THEN UPDATE SET
  tgt.customer_id = src.customer_id,
  tgt.customer_name = src.customer_name,
  tgt.customer_country = src.customer_country,
  tgt.customer_region = src.customer_region,
  tgt.customer_country_code = src.customer_country_code,
  tgt.sales_rep_id = src.sales_rep_id,
  tgt.sales_rep_name = src.sales_rep_name,
  tgt.sales_rep_country = src.sales_rep_country,
  tgt.sales_rep_region = src.sales_rep_region,
  tgt.sales_rep_country_code = src.sales_rep_country_code,
  tgt.product_code = src.product_code,
  tgt.product_details = src.product_details,
  tgt.unit_quantity = src.unit_quantity,
  tgt.sale_amount = src.sale_amount,
  tgt.sale_datetime = src.sale_datetime,
  tgt.sale_date = src.sale_date,
  tgt.sale_location = src.sale_location,
  tgt.currencycode = src.currencycode,
  tgt.sales_country_code = src.sales_country_code
WHEN NOT MATCHED THEN INSERT 
(
  sale_transaction_id, customer_id, customer_name, customer_country,
  customer_region, customer_country_code,
  sales_rep_id, sales_rep_name, sales_rep_country, sales_rep_region, sales_rep_country_code,
  product_code, product_details, unit_quantity, sale_amount,
  sale_datetime, sale_date, sale_location, currencycode, sales_country_code
)
VALUES (
  src.sale_transaction_id, src.customer_id, src.customer_name, src.customer_country,
  src.customer_region, src.customer_country_code,
  src.sales_rep_id, src.sales_rep_name, src.sales_rep_country, src.sales_rep_region, src.sales_rep_country_code,
  src.product_code, src.product_details, src.unit_quantity, src.sale_amount,
  src.sale_datetime, src.sale_date, src.sale_location, src.currencycode, src.sales_country_code
);

""")

df_rebates_silver.createOrReplaceTempView("rebates_increment")

spark.sql("""
MERGE INTO silver_fact_rebates_stage tgt
USING rebates_increment src
ON tgt.rebate_transaction_id = src.rebate_transaction_id
WHEN MATCHED THEN UPDATE SET
  tgt.sale_transaction_id = src.sale_transaction_id,
  tgt.customer_id = src.customer_id,
  tgt.sales_rep_id = src.sales_rep_id,
  tgt.rebate_type = src.rebate_type,
  tgt.rebate_amount = src.rebate_amount,
  tgt.rebate_datetime = src.rebate_datetime,
  tgt.rebate_date = src.rebate_date,
  tgt.currencycode = src.currencycode,
  tgt.sale_location = src.sale_location
WHEN NOT MATCHED THEN INSERT (
  rebate_transaction_id,
  sale_transaction_id,
  customer_id,
  sales_rep_id,
  rebate_type,
  rebate_amount,
  rebate_datetime,
  rebate_date,
  currencycode,
  sale_location
)
VALUES (
  src.rebate_transaction_id,
  src.sale_transaction_id,
  src.customer_id,
  src.sales_rep_id,
  src.rebate_type,
  src.rebate_amount,
  src.rebate_datetime,
  src.rebate_date,
  src.currencycode,
  src.sale_location
)

""")


StatementMeta(, d172367e-f2ce-4acf-ab47-0253266cc505, 24, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]